In [1]:
import urllib.request
import json
import pandas as pd

propertyid = "Property:P856"
url = (
    "https://www.wikidata.org/w/api.php?action=query&prop=revisions&titles="
    + propertyid
    + "&rvprop=timestamp|comment|ids&rvlimit=500&format=json"
)
response = urllib.request.urlopen(url)
responsedata = json.loads(response.read().decode("utf-8"))

In [2]:
array = []

for data in responsedata["query"]["pages"]:
    for revision in responsedata["query"]["pages"][data]["revisions"]:
        print(revision.keys())
        break

dict_keys(['revid', 'parentid', 'timestamp', 'comment'])


In [3]:
result = list(responsedata["query"]["pages"].values())[0]
revisions = result["revisions"]
revision = result["revisions"][0].keys()
revision

dict_keys(['revid', 'parentid', 'timestamp', 'comment'])

In [4]:
array = []

for revision in revisions:
    array.append([revision["timestamp"], revision["comment"]])

In [5]:
array[:1]

[['2025-02-11T14:11:42Z',
  '/* wbsetaliases-remove:1|en-gb */ official homepage']]

In [6]:
dataframe = pd.DataFrame(array, columns=["timestamp", "modification"])
dataframe['timestamp'] = pd.to_datetime(dataframe['timestamp'], format='%Y-%m-%dT%H:%M:%SZ')
dataframe = dataframe.astype(
    dtype={"modification": "<U1000"}
)

dataframe["timestamp"] = pd.to_datetime(
    dataframe["timestamp"],
    format="%d/%m/%Y",
    unit="D",
    errors="raise",
).dt.normalize()
dataframe

,timestamp,modification
0,2025-02-11,/* wbsetaliases-remove:1|en-gb */ official hom...
1,2025-02-11,"/* wbsetaliases-remove:9|en-ca */ website, hom..."
2,2025-02-11,/* wbsetdescription-remove:1|en-gb */ URL of t...
3,2025-02-11,/* wbsetdescription-remove:1|en-ca */ URL of t...
4,2025-02-11,/* wbsetlabel-remove:1|en-ca */ official website
...,...,...
495,2020-12-02,/* wbeditentity-update-languages-short:0||fa */
496,2020-11-24,"/* wbsetaliases-update:7|fr */ URL officielle,..."
497,2020-11-22,/* undo:0||1311009051|마인드 */
498,2020-11-22,/* wbsetclaim-update:2||1|1 */ [[Property:P230...


In [7]:
group = dataframe.groupby("timestamp").agg("count").reset_index()
group

,timestamp,modification
0,2020-11-07,1
1,2020-11-22,2
2,2020-11-24,1
3,2020-12-02,1
4,2020-12-15,7
...,...,...
231,2024-12-24,7
232,2024-12-30,6
233,2025-01-02,1
234,2025-01-31,1


In [8]:
group.describe()

,timestamp,modification
count,236,236.000000
mean,2022-12-28 08:38:38.644067840,2.118644
min,2020-11-07 00:00:00,1.000000
25%,2021-12-28 12:00:00,1.000000
50%,2022-11-22 12:00:00,1.000000
75%,2024-01-20 00:00:00,2.000000
max,2025-02-11 00:00:00,64.000000
std,NaN,4.496182


In [ ]:
group.dtypes

In [ ]:
group.modification.max()

In [ ]:
group.modification.min()

In [ ]:
import calplot

days = group["timestamp"]

events = pd.Series(list(group["modification"]), index=days)
calplot.calplot(
    events,
    cmap="YlGn",
    yearlabel_kws={"color": "black"},
    linewidth=0,
    suptitle="Number of modifications per day (last 500)",
)

In [ ]:
import plotly.express as px

fig = px.histogram(
    group,
    x="timestamp",
    y="modification",
    nbins=20,
    labels={"x": "month", "y": "count"},
)
fig.update_layout(bargap=0.2)
fig.show()

In [ ]:
import plotly.express as px

fig = px.scatter(
    group, x="timestamp", y="modification", labels={"x": "month", "y": "count"}
)
fig.update_layout(bargap=0.2)
fig.show()

In [ ]:
import plotly.express as px

fig = px.histogram(group, x="modification", histnorm="probability density", nbins=20)
fig.update_layout(bargap=0.2)
fig.show()

In [ ]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?heritagesite ?label ?coordinates {
  ?heritagesite wdt:P1435 wd:Q9259;
        wdt:P17 wd:Q142;
        rdfs:label ?label;
        wdt:P625 ?coordinates.
        FILTER (lang(?label)="en")
}
LIMIT 100
"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

array = []
for result in results["results"]["bindings"]:
    array.append(
        [
            result["heritagesite"]["value"],
            result["label"]["value"],
            result["coordinates"]["value"],
        ]
    )

In [ ]:
from shapely import wkt
import geopandas as gp

dataframe = pd.DataFrame(array, columns=["ID", "label", "coordinates"])
dataframe["coordinates"] = dataframe["coordinates"].apply(wkt.loads)

geodataframe = gp.GeoDataFrame(dataframe, geometry="coordinates")

In [ ]:
array[:1]

In [ ]:
geodataframe.dtypes

In [ ]:
import folium

map = folium.Map(location=[45.1093, 2.6753], zoom_start=6)
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in geodataframe.geometry]

In [ ]:
i = 0
for coordinates in geo_df_list:
    map.add_child(
        folium.Marker(
            location=coordinates,
            icon=folium.Icon(color="green"),
            popup="Wikidata: <a target='_blank' href='"
            + str(geodataframe.ID[i])
            + "'>"
            + str(geodataframe.label[i]),
        )
    )
    i = i + 1

map